XLM-Roberta on multilingual text

- Single Loss - on Overall only
- Parameter file: XLM_Roberta_1.00_overall_loss.pth

0. Install and import libraries

In [1]:
## install libraries

!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.2 MB/s 
     |████████████████████████████████| 6.6 MB 28.2 MB/s 
     |████████████████████████████████| 86 kB 2.9 MB/s 
     |████████████████████████████████| 596 kB 27.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 9.4 MB/s 


In [3]:
import pandas as pd
import numpy as np
import random
import json
import regex as re

import matplotlib.pyplot as plt
import os
import sentencepiece
import torch
import torch.nn as nn
from transformers import AutoTokenizer
from transformers import AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from transformers import XLMRobertaConfig, XLMRobertaModel, XLMRobertaTokenizer

torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

from google.colab import drive
drive.mount('/content/drive')
LIBRARY_PATH = '/content/drive/MyDrive/NLP PROJECT/Finals/'

# Seed
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

Mounted at /content/drive


## 1. Read and prepare data for training

In [4]:
def clean_text(text, fixed_length = True, length = 256, head_len = 200, tail_len = 56):
  text = re.sub(r'http\S+', '', text)
  text = re.sub("\n|\r", " ", text)
  text = re.sub("['']", " ", text)

  if(fixed_length == True):
    tokens = text.split()
    if(len(tokens) > length):
      head = tokens[:head_len]
      tail = tokens[-tail_len:]
      text = ' '.join(head+tail)

  return text


In [5]:
raw_data_path = LIBRARY_PATH + 'data/processed/'
raw_data_filename = 'train_DA.csv'
train_data = pd.read_csv(raw_data_path + raw_data_filename)


In [6]:
## Split into train and eval

# remove na text from both the text fields
processed_data = train_data[train_data['text1'].notna()]
processed_data = processed_data[processed_data['text2'].notna()]

print("After removing NA text columns, we lose {0} rows".format(train_data.shape[0] - processed_data.shape[0]))

After removing NA text columns, we lose 95 rows


In [7]:
def merge_clean_columns(df):
    """
    Merge multiple columns into one and clean text
    """  
    df['merge1'] = df['meta_keywords1'].astype(str) + ', ' \
        + df['meta_description1'].astype(str) + ', '\
        + df['title1'].astype(str) + ', '\
        + df['text1'].astype(str)

    df['merge2'] = df['meta_keywords2'].astype(str) + ', ' \
        + df['meta_description2'].astype(str) + ', '\
        + df['title2'].astype(str) + ', '\
        + df['text2'].astype(str)

    df['merge1'] = df['merge1'].apply(lambda x: clean_text(x))
    df['merge2'] = df['merge2'].apply(lambda x: clean_text(x))

    return df

In [8]:
processed_data = merge_clean_columns(processed_data)
# split into train and development
train, dev = train_test_split(processed_data, test_size=0.1, random_state = 42)

## 2. Model on data text

In [9]:
## set parameters
max_len = 512
batch_size = 8
lr = 5e-6
weight_decay = 1e-4
num_epochs = 8

In [10]:
def get_data_loader(data, batch_size_flg = True):
  tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
  input_ids, attention_masks, labels = [], [], []
  for idx, row in data.iterrows():
      text1, text2 = row['merge1'], row['merge2']
      encode_dict = tokenizer(text1,text2,
                                  max_length=max_len,
                                  padding='max_length',
                                  truncation=True,
                                  add_special_tokens=True
                                  )
      
      input_ids.append(encode_dict['input_ids'])
      attention_masks.append(encode_dict['attention_mask'])
      labels.append(float(row['Overall']))

  input_ids = torch.tensor(input_ids)
  attention_masks = torch.tensor(attention_masks)
  labels = torch.tensor(labels)

  data = TensorDataset(input_ids, attention_masks, labels)
  if(batch_size_flg):
      data_loader = DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)
  else:
      data_loader = DataLoader(data)
  return data_loader

In [ ]:
train_data_loader = get_data_loader(train)
eval_data_loader = get_data_loader(dev, False)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [11]:
class Custom_XLMRoberta(nn.Module):
    def __init__(self, model, hidden_size):
        super(Custom_XLMRoberta, self).__init__()
        self.reg_model = model
        self.fc1 = nn.Linear(hidden_size, 100)
        self.fc2 = nn.Linear(100, 1) ## currently processes the 1 label (Overall) only
        self.activation = nn.GELU()

    def forward(self, input_ids, attention_masks):
        output1 = self.reg_model(input_ids, attention_masks)[1]
        logits1 = self.fc2(self.activation(self.fc1(output1)))
        
        return logits1


In [12]:
def predict(model, data_loader):
  model.eval()
  overall_pred, overall_true = [], []
  with torch.no_grad():
    for idx, (ids, att_msks, y) in enumerate(data_loader):
      ids, att_msks, y = ids.to(device), att_msks.to(device), y.to(device)
      y_pred = model(ids, att_msks)
      y_pred, y =torch.squeeze(y_pred).cpu().numpy().tolist(), torch.squeeze(y).cpu().numpy().tolist()
      overall_pred.append(y_pred)
      overall_true.append(y)
  return overall_pred, overall_true


def train(model, model_path, train_data_loader, eval_data_loader, optimizer, epochs):
  model.train()
  criterion = nn.MSELoss()
  best_pearson = 0
  for i in range(epochs):
    train_loss_sum = 0
    for idx, (ids, att_msks, y) in enumerate(train_data_loader):
      ids, att_msks, y = ids.to(device), att_msks.to(device), y.to(device)
      optimizer.zero_grad()
      y_pred = model(ids, att_msks)
      y_pred, y = torch.squeeze(y_pred), torch.squeeze(y) ## not required when using 1 label
      loss = criterion(y_pred, y) # not using weighted loss yet
      loss.backward()
      optimizer.step()
      train_loss_sum += loss.item()

    print(f"Loss at epoch {i}: {train_loss_sum:.4f}")

    ## Determine best epoch model using correlation coefficient for Overall in dev data
    eval_pred_overall, eval_true_overall = predict(model, eval_data_loader)
    curr_pearson = np.corrcoef(eval_pred_overall, eval_true_overall)[0][1]
    print(curr_pearson)
    if curr_pearson > best_pearson:
      best_pearson = curr_pearson
      torch.save(model.state_dict(), model_path)
    


### Training model

In [ ]:
## run model finetuning and save fine-tuned model
# pre_trained_model = XLMRobertaModel.from_pretrained("xlm-roberta-large")
torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

pre_trained_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")
config = XLMRobertaConfig.from_pretrained("xlm-roberta-base")
hidden_size = config.hidden_size
# hidden_size = 768

model = Custom_XLMRoberta(pre_trained_model, hidden_size)
model.to(device)


for iter in range(1,1000):
    model_name = 'XLM_Roberta_base_Overall_loss'
    model_name_iter = f"{model_name}_iter_{iter}.pth"
    model_path = f"/content/drive/MyDrive/NLP PROJECT/Finals/Model/{model_name_iter}"
    if not os.path.exists(model_path):
      break

print(f"Model name for this run: {model_name_iter}")

optimizer = AdamW(model.parameters(), lr=lr, weight_decay = weight_decay)
train(model, model_path, train_data_loader, eval_data_loader, optimizer, num_epochs)


Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model name for this run: XLM_Roberta_base_Overall_loss_iter_1.pth


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Loss at epoch 0: 1649.7571
0.5616254667686394
Loss at epoch 1: 543.8352
0.7933985250671425
Loss at epoch 2: 329.7126
0.8418097215197305
Loss at epoch 3: 214.3843
0.8500608675706589
Loss at epoch 4: 137.3386
0.8592470288585194
Loss at epoch 5: 91.0931
0.8579712704597856
Loss at epoch 6: 63.8091
0.8711330591209295
Loss at epoch 7: 50.4867
0.8747507371702355


## 4. Evaluation on test data

In [13]:
#### Get data
data_path = LIBRARY_PATH + 'data/processed/'
filename = 'paired_eval.csv'
path = data_path + filename
tmp_file = pd.read_csv(path)

tmp_file_new = tmp_file.drop_duplicates()
# Drop na for text 1 and text 2
test_dropna_text1 = tmp_file_new[tmp_file_new['text1'].notna()]
test_dropna_text1_2 = test_dropna_text1[test_dropna_text1['text2'].notna()]
# Merge data
processed_test_data = merge_clean_columns(test_dropna_text1_2)

test_data_loader = get_data_loader(processed_test_data, False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [15]:
config = XLMRobertaConfig.from_pretrained("xlm-roberta-base")
hidden_size = config.hidden_size

pre_trained_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")
model = Custom_XLMRoberta(pre_trained_model, hidden_size)
#model.load_state_dict(torch.load(model_path))
model.load_state_dict(torch.load("/content/drive/MyDrive/NLP PROJECT/Finals/ModelParams/XLM_Roberta_1.00_overall_loss.pth"), strict=False)
model.to(device)

test_pred_overall, test_true_overall = predict(model, test_data_loader)
test_pearson_score = np.corrcoef(test_pred_overall, test_true_overall)[0][1]

print("Pearson score on test dataset is {:.3f}".format(test_pearson_score))


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Pearson score on test dataset is 0.723
